# Introductie

In dit notebook leer je stap voor stap hoe je een decision tree kan implementeren. Er zijn een hoop manieren, maar we kijken vandaag naar een implementatie van Jason Brownlee, schrijver van o.a. Python en Deep Learning boeken.

We gebruiken de Titanic dataset, en ons doel is om te voorspellen of een persoon de ramp heeft overleefd op basis van een aantal eigenschappen.

We doen de implementatie op basis van numpy, een python package voor razendsnelle berekeningen.

In [394]:
# dit zijn alle imports en voorbereiding van de dataset, hier hoef je niks aan te veranderen.

import seaborn as sns
from random import seed
from random import randrange
from csv import reader
import numpy as np

columns = ["pclass", "sex", "age", "deck", "alone", "survived"]

df = sns.load_dataset('titanic')[columns]
df["age"] = df["age"].fillna(np.mean(df["age"])).astype(int)
df["deck"] = df["deck"].astype(str).replace("nan", "O")
df["pclass"] = df["pclass"].astype(str).replace("nan", "O")

print(df.columns)
train = df[:-100].values
test = df[-100:].values

Index(['pclass', 'sex', 'age', 'deck', 'alone', 'survived'], dtype='object')


# Gini Index

Als eerste hebben we een metric nodig die aangeeft hoe goed een split in de decision tree is. We gebruiken hiervoor de gini index. Hier is p1 de kans op class 1, en p2 de kans op class 2.

$$
1 – (p₁)² – (p₂)²
$$


We kunnen dit als volgt implementeren:

1. bereken voor elke class het percentage hoevaak deze voorkomt.
2. doe elk percentage in het kwadraat en tel dit bij elkaar op.

BONUS: bedenk een implementatie die niet voor 2 classes, maar voor *n* classes werkt.

In [274]:
def gini_index(labels):

  # bereken het percentage hoevaak elke class voorkomt in data
  p = [np.count_nonzero(labels == item) / len(labels) for item in set(labels)]  # bevat een lijst met een percentage voor elke class.

  # doe elk percentage in het kwadraat en tel dit allemaal bij elkaar op.
  pi = sum([percentage ** 2 for percentage in p])

  return 1 - pi # lager is beter

print(gini_index(train[:,-1]))

0.47439509910002053


# Weighted Gini Index

We kunnen de data splitsen in twee groepen door een eigenschap te kiezen en een waarde van die eigenschap. Bijvoorbeeld 'geslacht = man' of 'leeftijd < 18'.

Als je in 1 groep alle overlevenden hebt, en in de andere alle niet overlevenden heb je volgens de gini index een perfecte score van 0. Als je split nog steeds willekeurig is dan heb je een score van 0.5 (in het geval van een binair probleem).

We willen nu niet de gini index van een split weten, maar de gewogen gini index van twee splits.

Dit kunnen we als volgt doen:
1. Bereken de gini index voor beide groepen.
2. Bereken de relatieve grootte voor beide groepen (als je een groep van 4 en een groep van 16 hebt, is het 0.2 en 0.8).
3. Vermenigvuldig de gini index met de relatieve grootte en tel deze bij elkaar op.

BONUS: voeg iets toe dat voorkomt dat we per ongeluk door 0 delen. Als een groep leeg is geef dan een perfecte score.

In [275]:
def weighted_gini_index(group_a, group_b):
  n_instances = len(group_a) + len(group_b)

  gini = 0.0
  for group in (group_a, group_b):
    gini += gini_index(group) * (len(group) / n_instances)

  return gini


print(weighted_gini_index(train[:200,-1], train[200:, -1]))

0.4732095849884808


# Een split maken

Het splitsen van data doen we op basis van een eigenschap en een waarde van die eigenschap. We hebben numerieke eigenschappen (leeftijd) en categorieen (geslacht).

Implementeer eerst een stukje code voor numerieke eigenschappen. De data die lager is dan de opgegeven waarde moet naar links, en hoger naar rechts.

In [276]:
def test_split(data, feature_index, value):
  mask = data[:,feature_index] < value

  return data[mask], data[~mask]

print(test_split(train[:20], 2, 20))

(array([['3', 'male', 2, 'O', False, 0],
       ['2', 'female', 14, 'O', False, 1],
       ['3', 'female', 4, 'G', False, 1],
       ['3', 'female', 14, 'O', True, 0],
       ['3', 'male', 2, 'O', False, 0]], dtype=object), array([['3', 'male', 22, 'O', False, 0],
       ['1', 'female', 38, 'C', False, 1],
       ['3', 'female', 26, 'O', True, 1],
       ['1', 'female', 35, 'C', False, 1],
       ['3', 'male', 35, 'O', True, 0],
       ['3', 'male', 29, 'O', True, 0],
       ['1', 'male', 54, 'E', True, 0],
       ['3', 'female', 27, 'O', False, 1],
       ['1', 'female', 58, 'C', True, 1],
       ['3', 'male', 20, 'O', True, 0],
       ['3', 'male', 39, 'O', False, 0],
       ['2', 'female', 55, 'O', True, 1],
       ['2', 'male', 29, 'O', True, 1],
       ['3', 'female', 31, 'O', False, 0],
       ['3', 'female', 29, 'O', True, 1]], dtype=object))


Upgrade nu datzelfde stukje code. Als het type van value een string is, dan moet de data die die value heeft naar links, en de rest naar rechts.

In [277]:
def test_split(data, feature_index, value):
  if type(value) is str:
    mask = data[:,feature_index] == value
  else:
    mask = data[:,feature_index] < value


  return data[mask], data[~mask]

print(test_split(train[:10], 2, 15))
print(test_split(train[:10], 1, "female"))

(array([['3', 'male', 2, 'O', False, 0],
       ['2', 'female', 14, 'O', False, 1]], dtype=object), array([['3', 'male', 22, 'O', False, 0],
       ['1', 'female', 38, 'C', False, 1],
       ['3', 'female', 26, 'O', True, 1],
       ['1', 'female', 35, 'C', False, 1],
       ['3', 'male', 35, 'O', True, 0],
       ['3', 'male', 29, 'O', True, 0],
       ['1', 'male', 54, 'E', True, 0],
       ['3', 'female', 27, 'O', False, 1]], dtype=object))
(array([['1', 'female', 38, 'C', False, 1],
       ['3', 'female', 26, 'O', True, 1],
       ['1', 'female', 35, 'C', False, 1],
       ['3', 'female', 27, 'O', False, 1],
       ['2', 'female', 14, 'O', False, 1]], dtype=object), array([['3', 'male', 22, 'O', False, 0],
       ['3', 'male', 35, 'O', True, 0],
       ['3', 'male', 29, 'O', True, 0],
       ['1', 'male', 54, 'E', True, 0],
       ['3', 'male', 2, 'O', False, 0]], dtype=object))


# De beste split vinden

Hierboven hebben we alle ingredienten voorbereid om de beste split te vinden. We moeten voor de hele dataset elke mogelijke split identificeren, deze evalueren, en uiteindelijk de beste opslaan.

Efficient? Nee. Effectief? Ja.

Een split representeren we als een dictionary met de volgende keys:
1. de index van de feature.
2. de waarde waar we gaan splitsen.
3. de groepen na de split.

Implementeer hieronder de get split functie. Ik heb hieronder het algoritme omschreven:

```
Voor elke feature:
  Identificeer elke mogelijke split optie.
  Voor elke split optie:
    Maak een split.
    Bereken de weighted gini.
    Als de gini score de beste tot nu toe is:
      Sla de waardes voor de split op.
      Continue loop
```




In [332]:
def get_split(dataset):
  best_score = 999
  best_value = None
  best_index = None
  best_groups = None
  best_right = None

  for i in range(0, dataset.shape[1] -1):
    options = np.unique(dataset[:,i])
    for val in options:
      a, b = test_split(dataset, i, val)
      gini = weighted_gini_index(a[:,-1], b[:,-1])

      if gini < best_score:
        best_score = gini
        best_value = val
        best_index = i
        best_groups = a, b

  return {'feature_index': best_index, 'value': best_value, 'groups': best_groups, 'gini': best_score}

get_split(train[:10])

{'feature_index': 1,
 'value': 'female',
 'groups': (array([['1', 'female', 38, 'C', False, 1],
         ['3', 'female', 26, 'O', True, 1],
         ['1', 'female', 35, 'C', False, 1],
         ['3', 'female', 27, 'O', False, 1],
         ['2', 'female', 14, 'O', False, 1]], dtype=object),
  array([['3', 'male', 22, 'O', False, 0],
         ['3', 'male', 35, 'O', True, 0],
         ['3', 'male', 29, 'O', True, 0],
         ['1', 'male', 54, 'E', True, 0],
         ['3', 'male', 2, 'O', False, 0]], dtype=object)),
 'gini': 0.0}

# De decision tree

Het is ons net gelukt om onze eerste split te doen, nu gewoon dit herhalen toch?

Om dit te blijven herhalen tot we klaar zijn hebben we een recursief algoritme nodig, dit is in essentie een functie die zichzelf aanroept tot het doel is bereikt.

We hebben ons doel bereikt als we geen verdere split meer kan maken. Dit kan om twee redenen:
1. Als er minder dan twee datapunten zijn.
2. De hele dataset hoort bij een enkele class (gini score van 0).

Als we dit doel hebben bereikt willen we een 'leaf node' maken die aangeeft welke class moet worden gekozen.


We weten nu wanneer we moeten stoppen en wat we dan moeten doen, dus laten we onze 'to leaf' functie implementeren.

De 'to leaf' functie moet de meest voorkomende class in de dataset teruggeven.

Houd in je achterhoofd dat in de dataset de laatste kolom de class bevat.

In [358]:
def to_leaf(dataset):
  u, c = np.unique(dataset[:,-1], return_counts=True)
  res = u[c.argmax()]
  return res

print(to_leaf(train[:50]))

0


Laten we nu de 'grow tree' functie schrijven. Dit wordt een recursieve functie, wat dus betekent dat we in de 'grow tree' functie NOG een keer de 'grow tree' functie gaan aanroepen.

Het grow tree algoritme moet de volgende stappen gaan nemen:

1. Controleer of 1 van de twee datasets in de node leeg is, maak dan een terminal node van beide kanten met de gecombineerde data.
2. Als de linker dataset minder dan twee datapunten heeft, maak dan een terminal node van de linker dataset.
3.Als de linker dataset twee of meer datapunten heeft, maak dan weer een nieuwe split met 'get_split' en groei op het resultaat je decision tree verder.
4. Doe stap 2 en 3 ook voor de rechter dataset.

Als de node leeg is kan je stoppen met de recursie, dat doe je met behulp van een lege return (dan stopt de functie).

Naast alles wat we hierboven gezegd hebben gaan we ook de diepte van de tree bijhouden. Dit kan heel simpel, elke keer als je 'grow_tree' opnieuw aanroept, geeft dan de huidige diepte + 1 mee. Dit is handig om later de tree te visualiseren.

In [384]:
from token import RIGHTSHIFT
# Build a decision tree
def grow_tree(node, depth):
    left, right = node["groups"]

    if len(left) == 0:
      node["left"] = node["right"] = to_leaf(right)
      return
    if len(right) == 0:
      node["left"] = node["right"] = to_leaf(left)
      return

    if len(left) > 2:
        node["left"] = get_split(left)
        grow_tree(node["left"], depth+1)
    else:
        node["left"] = to_leaf(left)

    if len(right) > 2:
        node["right"] = get_split(right)
        grow_tree(node["right"], depth+1)
    else:
        node["right"] = to_leaf(right)

node = get_split(train[:10])
tree = grow_tree(node, 0)
print(tree)

None


Het is je gelukt! Je decision tree kan nu naar hartelust groeien. Onderstaande code roept je algoritme aan op de test set om te kijken hoe accuraat je modelletje is. Je kan ook (ongeveer) zien hoe de tree in elkaar zit.

In [393]:
def decision_tree(train):
 root = get_split(train)
 grow_tree(root, depth=0)
 return root

# Print a decision tree
def print_tree(node,col_table, depth=1):
  if isinstance(node, dict):

    col_name = col_table[node['feature_index']]

    print(" "  * depth + f"{col_name , node['value']}")
    print_tree(node['left'], col_table, depth+1)
    print_tree(node['right'], col_table, depth+1)
  else:
    print(" " * depth + "class: ", node)

tree = decision_tree(train[0:50])
print_tree(tree, columns, depth=1)

 ('sex', 'female')
  ('alone', True)
   ('deck', 'O')
    ('age', 18)
     ('pclass', '2')
      class:  1
      class:  0
     ('age', 27)
      class:  0
      ('pclass', '2')
       class:  0
       ('age', 31)
        class:  1
        ('age', 38)
         class:  0
         class:  0
    ('pclass', '1')
     ('pclass', '1')
      class:  1
      class:  1
     class:  1
   ('age', 15)
    class:  0
    ('pclass', '1')
     class:  1
     ('pclass', '2')
      class:  1
      ('pclass', '3')
       class:  1
       class:  1
  ('deck', 'A')
   class:  1
   ('deck', 'D')
    class:  1
    ('pclass', '2')
     ('age', 35)
      class:  1
      class:  0
     ('alone', True)
      ('pclass', '1')
       ('pclass', '1')
        class:  0
        class:  0
       ('pclass', '3')
        class:  0
        class:  0
      ('age', 35)
       ('age', 29)
        class:  0
        ('pclass', '3')
         class:  0
         class:  0
       ('pclass', '1')
        class:  0
        class:  0

Als je goed kijkt kan je hier zien dat we heel vaak splits maken die onnodig zijn (namelijk als beide classes hetzelfde zijn). Zie onderaan de bonus opdrachten om hier wat aan te doen.

Hier onder staat de code om je gemaakte tree te evalueren, doe je het beter dan een willekeurige gok (50%)?

In [389]:
def predict(node, row):
  go_left = None
  if type(node['value']) is str:
    go_left = row[node['feature_index']] == node['value']
  else:
    go_left = row[node['feature_index']] < node['value']

  if go_left:
    if isinstance(node['left'], dict):
      return predict(node['left'], row)
    else:
      return node['left']
  else:
    if isinstance(node['right'], dict):
      return predict(node['right'], row)
    else:
      return node['right']


def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

def evaluate_algorithm(tree, test):
  actual = list(test[:,-1])
  predictions = list()
  for row in test:
    prediction = predict(tree, row)
    predictions.append(prediction)

  return accuracy_metric(actual, predictions)


tree = decision_tree(train[:300])


print(f"Je hebt een score van {evaluate_algorithm(tree, test)}% behaald, is dat beter dan een willekeurige gok? Dan ben je geslaagd!")


Je hebt een score van 80.0% behaald, is dat beter dan een willekeurige gok? Dan ben je geslaagd!
